In [11]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [12]:
!pip install playwright

In [3]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as TimeoutPlaywright
import time
import pandas as pd

In [4]:
# All NBA Seasons that will be scraped for data (2016 - 2024)
NBA_SEASONS = list(range(2016, 2025))
NBA_SEASONS

[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [5]:
# Directory to store all data
DATA_DIRECTORY = "nba_data"

# Directory to store all standings (list out box score data)
STANDINGS_DIRECTORY = os.path.join(DATA_DIRECTORY, "standings")

# Directory to store all box scores
SCORES_DIRECTORY = os.path.join(DATA_DIRECTORY, "scores")

In [6]:
# Retrieve HTML with 5 possible retries
async def retrieve_html(url, selector, sleep=5, retries=5):
    html = None
    for i in range(1, retries+1):
        # Exponential backoff so web scraping doesn't occur too fast
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                launch_browser = await p.chromium.launch()
                new_browser_tab = await launch_browser.new_page()
                await new_browser_tab.goto(url)
                html = await new_browser_tab.inner_html(selector)
        # When there is an issue scraping the page:
        except TimeoutPlaywright:
            print(f"URL {url} caused TimeoutError")
            continue
        # Successful scrape:
        else: 
            break
    return html

In [7]:
# Function to scrape a single season's page (parameter: season)
async def scrape_seasonpage(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    
    # Pass URL through CSS Selector
    html = await retrieve_html(url, "#content .filter")
    
    if not html:
        print(f"Failed to retrieve season page for {season}")
        return
    
    # Grab all <a> elements and their href properties
    soup = BeautifulSoup(html)
    a_tags = soup.find_all("a")
    all_hrefs = [link["href"] for link in a_tags]

    # Turn hrefs into full URLs
    full_links = [f"https://basketball-reference.com{link}" for link in all_hrefs]
    
    for link in full_links:
        save_to_dir = os.path.join(STANDINGS_DIRECTORY, link.split("/")[-1])
        # Avoid repeated saving
        if os.path.exists(save_to_dir):
            continue
        
        # Get table with box score from all href links
        html = await retrieve_html(link, "#all_schedule")
        
        if not html:
            print(f"Failed to retrieve data from {link}")
            continue
                  
        # Save HTML
        with open(save_to_dir, "w+") as f:
            f.write(html)
        print(f"Saved data for {link}")

In [8]:
for year in NBA_SEASONS:
    await scrape_seasonpage(year)

In [9]:
standings = os.listdir(STANDINGS_DIRECTORY)
standings

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2024_games-may.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2023_games-may.html',
 'NBA_2019_games-april.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2016_games-april.html',
 'NBA_2024_games-march.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2024_games-november.html',
 'NBA_2017_games-february.html',
 'NBA_2024_games-december.html',
 'NBA_2016_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-november.html',

In [10]:
# Process tables to get individual box scores
async def scrape_individual_game(standings):
    with open(standings, 'r') as f:
        html = f.read()
        
    # Similar process as scrape_seasons function
    soup = BeautifulSoup(html)
    a_tags = soup.find_all("a")
    all_hrefs = [link.get('href') for link in a_tags]
    
    box_scores = [f"https://basketball-reference.com{link}" for link in all_hrefs if link and "boxscore" in link and '.html' in link]
    
    for score in box_scores:
        save_to_dir = os.path.join(SCORES_DIRECTORY, score.split("/")[-1])
        # Avoid repeated saving
        if os.path.exists(save_to_dir):
            continue
        
        # Get box scores
        html = await retrieve_html(score, "#content")
        
        if not html:
            print(f"Failed to retrieve score from {score}")
            continue
                  
        # Save HTML
        with open(save_to_dir, "w+") as f:
            f.write(html)
        print(f"Saved score for {score}")
    

In [11]:
# Optional Extension Season for if more seasons need to be added later
EXTENSION_SEASONS = list(range(2023, 2025))

for season in EXTENSION_SEASONS:
    print (f"Processing season: {season}")
    
    # Filters standings list to get files corresponding to current season
    files_per_season = [s for s in standings if str(season) in s]
    
    for f in files_per_season:
        new_path = os.path.join(STANDINGS_DIRECTORY, f)
        
        await scrape_individual_game(new_path)
        print(f"Finished processing file: {new_path}")
        
    print("Finished processing season {season}")

print("Finished processing all seasons.")

Processing season: 2023
Saved score for https://basketball-reference.com/boxscores/202305010BOS.html
Saved score for https://basketball-reference.com/boxscores/202305010DEN.html
Saved score for https://basketball-reference.com/boxscores/202305020NYK.html
Saved score for https://basketball-reference.com/boxscores/202305020GSW.html
Saved score for https://basketball-reference.com/boxscores/202305030BOS.html
Saved score for https://basketball-reference.com/boxscores/202305040GSW.html
Saved score for https://basketball-reference.com/boxscores/202305050PHI.html
Saved score for https://basketball-reference.com/boxscores/202305050PHO.html
Saved score for https://basketball-reference.com/boxscores/202305060MIA.html
Saved score for https://basketball-reference.com/boxscores/202305060LAL.html
Saved score for https://basketball-reference.com/boxscores/202305070PHI.html
Saved score for https://basketball-reference.com/boxscores/202305070PHO.html
Saved score for https://basketball-reference.com/box

Saved score for https://basketball-reference.com/boxscores/202304090OKC.html
Saved score for https://basketball-reference.com/boxscores/202304090PHO.html
Saved score for https://basketball-reference.com/boxscores/202304090POR.html
Saved score for https://basketball-reference.com/boxscores/202304110MIA.html
Saved score for https://basketball-reference.com/boxscores/202304110LAL.html
Saved score for https://basketball-reference.com/boxscores/202304120TOR.html
Saved score for https://basketball-reference.com/boxscores/202304120NOP.html
Saved score for https://basketball-reference.com/boxscores/202304140MIA.html
Saved score for https://basketball-reference.com/boxscores/202304140MIN.html
Saved score for https://basketball-reference.com/boxscores/202304150PHI.html
Saved score for https://basketball-reference.com/boxscores/202304150BOS.html
Saved score for https://basketball-reference.com/boxscores/202304150CLE.html
Saved score for https://basketball-reference.com/boxscores/202304150SAC.html

Saved score for https://basketball-reference.com/boxscores/202301080TOR.html
Saved score for https://basketball-reference.com/boxscores/202301080IND.html
Saved score for https://basketball-reference.com/boxscores/202301080MEM.html
Saved score for https://basketball-reference.com/boxscores/202301080MIA.html
Saved score for https://basketball-reference.com/boxscores/202301080HOU.html
Saved score for https://basketball-reference.com/boxscores/202301080OKC.html
Saved score for https://basketball-reference.com/boxscores/202301080PHO.html
Saved score for https://basketball-reference.com/boxscores/202301080LAC.html
Saved score for https://basketball-reference.com/boxscores/202301090WAS.html
Saved score for https://basketball-reference.com/boxscores/202301090BOS.html
Saved score for https://basketball-reference.com/boxscores/202301090NYK.html
Saved score for https://basketball-reference.com/boxscores/202301090MEM.html
Saved score for https://basketball-reference.com/boxscores/202301090DEN.html

Saved score for https://basketball-reference.com/boxscores/202301220PHO.html
Saved score for https://basketball-reference.com/boxscores/202301220GSW.html
Saved score for https://basketball-reference.com/boxscores/202301220POR.html
Saved score for https://basketball-reference.com/boxscores/202301230DET.html
Saved score for https://basketball-reference.com/boxscores/202301230ORL.html
Saved score for https://basketball-reference.com/boxscores/202301230CHI.html
Saved score for https://basketball-reference.com/boxscores/202301230HOU.html
Saved score for https://basketball-reference.com/boxscores/202301230UTA.html
Saved score for https://basketball-reference.com/boxscores/202301230POR.html
URL https://basketball-reference.com/boxscores/202301230SAC.html caused TimeoutError
Saved score for https://basketball-reference.com/boxscores/202301230SAC.html
Saved score for https://basketball-reference.com/boxscores/202301240IND.html
Saved score for https://basketball-reference.com/boxscores/202301240

Saved score for https://basketball-reference.com/boxscores/202211060LAL.html
Saved score for https://basketball-reference.com/boxscores/202211060MEM.html
Saved score for https://basketball-reference.com/boxscores/202211060TOR.html
Saved score for https://basketball-reference.com/boxscores/202211060LAC.html
Saved score for https://basketball-reference.com/boxscores/202211070CHO.html
Saved score for https://basketball-reference.com/boxscores/202211070ORL.html
Saved score for https://basketball-reference.com/boxscores/202211070DET.html
Saved score for https://basketball-reference.com/boxscores/202211070IND.html
Saved score for https://basketball-reference.com/boxscores/202211070PHI.html
Saved score for https://basketball-reference.com/boxscores/202211070ATL.html
Saved score for https://basketball-reference.com/boxscores/202211070MIA.html
Saved score for https://basketball-reference.com/boxscores/202211070CHI.html
Saved score for https://basketball-reference.com/boxscores/202211070MEM.html

...

Saved score for https://basketball-reference.com/boxscores/202402220OKC.html
Saved score for https://basketball-reference.com/boxscores/202402220DEN.html
Saved score for https://basketball-reference.com/boxscores/202402220UTA.html
Saved score for https://basketball-reference.com/boxscores/202402220GSW.html
Saved score for https://basketball-reference.com/boxscores/202402220SAC.html
Saved score for https://basketball-reference.com/boxscores/202402230ATL.html
Saved score for https://basketball-reference.com/boxscores/202402230PHI.html
Saved score for https://basketball-reference.com/boxscores/202402230HOU.html
Saved score for https://basketball-reference.com/boxscores/202402230MEM.html
Saved score for https://basketball-reference.com/boxscores/202402230NOP.html
Saved score for https://basketball-reference.com/boxscores/202402230OKC.html
Saved score for https://basketball-reference.com/boxscores/202402230GSW.html
Saved score for https://basketball-reference.com/boxscores/202402230MIN.html

Saved score for https://basketball-reference.com/boxscores/202401060HOU.html
Saved score for https://basketball-reference.com/boxscores/202401070CLE.html
Saved score for https://basketball-reference.com/boxscores/202401070BRK.html
Saved score for https://basketball-reference.com/boxscores/202401070ORL.html
Saved score for https://basketball-reference.com/boxscores/202401070SAC.html
Saved score for https://basketball-reference.com/boxscores/202401070DAL.html
Saved score for https://basketball-reference.com/boxscores/202401070DEN.html
Saved score for https://basketball-reference.com/boxscores/202401070PHO.html
Saved score for https://basketball-reference.com/boxscores/202401070GSW.html
Saved score for https://basketball-reference.com/boxscores/202401070LAL.html
Saved score for https://basketball-reference.com/boxscores/202401080CHO.html
Saved score for https://basketball-reference.com/boxscores/202401080IND.html
Saved score for https://basketball-reference.com/boxscores/202401080WAS.html

Saved score for https://basketball-reference.com/boxscores/202401210HOU.html
Saved score for https://basketball-reference.com/boxscores/202401210PHO.html
Saved score for https://basketball-reference.com/boxscores/202401210LAL.html
Saved score for https://basketball-reference.com/boxscores/202401220DET.html
Saved score for https://basketball-reference.com/boxscores/202401220ORL.html
Saved score for https://basketball-reference.com/boxscores/202401220PHI.html
Saved score for https://basketball-reference.com/boxscores/202401220TOR.html
Saved score for https://basketball-reference.com/boxscores/202401220MIN.html
Saved score for https://basketball-reference.com/boxscores/202401220DAL.html
Saved score for https://basketball-reference.com/boxscores/202401220PHO.html
Saved score for https://basketball-reference.com/boxscores/202401220SAC.html
Saved score for https://basketball-reference.com/boxscores/202401230IND.html
Saved score for https://basketball-reference.com/boxscores/202401230BRK.html

Saved score for https://basketball-reference.com/boxscores/202404030PHO.html
Saved score for https://basketball-reference.com/boxscores/202404040DAL.html
Saved score for https://basketball-reference.com/boxscores/202404040MIA.html
Saved score for https://basketball-reference.com/boxscores/202404040NYK.html
Saved score for https://basketball-reference.com/boxscores/202404040HOU.html
Saved score for https://basketball-reference.com/boxscores/202404040LAC.html
Saved score for https://basketball-reference.com/boxscores/202404050CHO.html
Saved score for https://basketball-reference.com/boxscores/202404050IND.html
Saved score for https://basketball-reference.com/boxscores/202404050WAS.html
Saved score for https://basketball-reference.com/boxscores/202404050BOS.html
Saved score for https://basketball-reference.com/boxscores/202404050CHI.html
URL https://basketball-reference.com/boxscores/202404050HOU.html caused TimeoutError
Saved score for https://basketball-reference.com/boxscores/202404050

Saved score for https://basketball-reference.com/boxscores/202404200NYK.html
Saved score for https://basketball-reference.com/boxscores/202404200DEN.html
Saved score for https://basketball-reference.com/boxscores/202404210BOS.html
Saved score for https://basketball-reference.com/boxscores/202404210LAC.html
Saved score for https://basketball-reference.com/boxscores/202404210MIL.html
Saved score for https://basketball-reference.com/boxscores/202404210OKC.html
Saved score for https://basketball-reference.com/boxscores/202404220CLE.html
Saved score for https://basketball-reference.com/boxscores/202404220NYK.html
Saved score for https://basketball-reference.com/boxscores/202404220DEN.html
Saved score for https://basketball-reference.com/boxscores/202404230MIN.html
Saved score for https://basketball-reference.com/boxscores/202404230MIL.html
Saved score for https://basketball-reference.com/boxscores/202404230LAC.html
Saved score for https://basketball-reference.com/boxscores/202404240BOS.html